In [25]:
import re
import os
from os import path
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

LINUX_COLS = ['i', 'rx_desc', 'rx_bytes', 'tx_desc', 'tx_bytes', 'instructions', 'cycles', 'ref_cycles', 'llc_miss', 'c0', 'c1', 'c1e', 'c3', 'c6', 'c7', 'joules', 'timestamp']

TIME_CONVERSION_khz = 1./(2899999*1000)
JOULE_CONVERSION = 0.00001526

# 50 100 200 400 500
itrs = ["0", "300", "1000"]

dvfss = ["0x0c00", "0x1d00"]

#iter = 5
## 75

In [9]:
def test(mqps):
    print(mqps)
    for dvfs in dvfss:
        for itr in itrs:
            print(f'/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate{mqps}_300000_fbuff-1_itr{itr}_dvfs{dvfs}_rapl135')
            for iter in range(0, 1):
                loc=f'/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate{mqps}_300000_fbuff-1_itr{itr}_dvfs{dvfs}_rapl135_repeat{iter}'
                num_interrupts = 0
                rxbytes = 0
                txbytes = 0

                tins = 0
                trefcyc = 0
                tllcm = 0
                tjoules = 0

                for core in range(0, 16):
                    fname = f'{loc}/ITRlogs/linux.flink.dmesg._'+str(core)+"_"+str(loc[len(loc)-1])
                    df = pd.read_csv(fname, sep=' ', names=LINUX_COLS)
                    rxbytes += df['rx_bytes'].sum()
                    txbytes += df['tx_bytes'].sum()
                    num_interrupts += len(df.index)
                    df_non0j = df[(df['joules']>0) 
                                  & (df['instructions'] > 0) 
                                  & (df['cycles'] > 0) 
                                  & (df['ref_cycles'] > 0) 
                                  & (df['llc_miss'] > 0)].copy()
                    df_non0j['timestamp'] = df_non0j['timestamp'] - df_non0j['timestamp'].min()
                    df_non0j['timestamp'] = df_non0j['timestamp'] * TIME_CONVERSION_khz
                    df_non0j['ref_cycles'] = df_non0j['ref_cycles'] * TIME_CONVERSION_khz
                    df_non0j['joules'] = df_non0j['joules'] * JOULE_CONVERSION

                    ## filters logs to be between 5min - 10 min
                    df_non0j = df_non0j[(df_non0j['timestamp'] > 60) & (df_non0j['timestamp'] < 240)]

                    tmp = df_non0j[['instructions', 'cycles', 'ref_cycles', 'llc_miss', 'joules', 'c0', 'c1', 'c1e', 'c3', 'c6', 'c7','timestamp']].diff()
                    tmp.columns = [f'{c}_diff' for c in tmp.columns]
                    df_non0j = pd.concat([df_non0j, tmp], axis=1)
                    df_non0j.dropna(inplace=True)
                    df.dropna(inplace=True)
                    df_non0j = df_non0j[df_non0j['joules_diff'] > 0]

                    tins += df_non0j['instructions_diff'].sum()
                    trefcyc += df_non0j['ref_cycles_diff'].sum()
                    tllcm += df_non0j['llc_miss_diff'].sum()
                    cjoules = df_non0j['joules_diff'].sum()
                    if core == 0 or core == 1:
                        tjoules += cjoules

                    #print(core, round(df_non0j['timestamp'].min(), 2), round(df_non0j['timestamp'].max(), 2), len(df.index))
                    #num_interrupts.append(len(df.index))
                #print(f'{iter}: {rxbytes} {txbytes} {int(tins)} {round(trefcyc, 2)} {int(tllcm)} {round(tjoules, 2)} Joules')
                print(f'{iter}: itr={itr} dvfs={dvfs} num_interrupts={num_interrupts} energy={round(tjoules, 2)} Joules')
                print("")
                #plt.scatter(range(0, 16), num_interrupts)
                #plt.title(loc)
                #plt.show()
                #print("")

In [10]:
test(200000)

200000
/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate200000_300000_fbuff-1_itr0_dvfs0x0c00_rapl135
0: itr=0 dvfs=0x0c00 num_interrupts=1734646 energy=6228.97 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate200000_300000_fbuff-1_itr300_dvfs0x0c00_rapl135
0: itr=300 dvfs=0x0c00 num_interrupts=479488 energy=6711.63 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate200000_300000_fbuff-1_itr1000_dvfs0x0c00_rapl135
0: itr=1000 dvfs=0x0c00 num_interrupts=295248 energy=6680.35 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate200000_300000_fbuff-1_itr0_dvfs0x1d00_rapl135
0: itr=0 dvfs=0x1d00 num_interrupts=1835926 energy=8247.17 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate200000_300000_fbuff-1_itr300_dvfs0x1d00_rapl135
0: itr=300 dvfs=0x1d00 num_interrupts=490059 energy=8469.75 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate200000_300000_fbuff-1_itr1000_dvfs0x1d0

In [13]:
test(400000)

400000
/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate400000_300000_fbuff-1_itr0_dvfs0x0c00_rapl135
0: itr=0 dvfs=0x0c00 num_interrupts=2988300 energy=6965.2 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate400000_300000_fbuff-1_itr300_dvfs0x0c00_rapl135
0: itr=300 dvfs=0x0c00 num_interrupts=889930 energy=6974.02 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate400000_300000_fbuff-1_itr1000_dvfs0x0c00_rapl135
0: itr=1000 dvfs=0x0c00 num_interrupts=563373 energy=6989.19 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate400000_300000_fbuff-1_itr0_dvfs0x1d00_rapl135
0: itr=0 dvfs=0x1d00 num_interrupts=3702664 energy=9717.47 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate400000_300000_fbuff-1_itr300_dvfs0x1d00_rapl135
0: itr=300 dvfs=0x1d00 num_interrupts=934363 energy=9903.79 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate400000_300000_fbuff-1_itr1000_dvfs0x1d00

In [14]:
print(9903/6974)

1.4199885288213363


In [16]:
print(8469/8247)

1.0269188795925792


In [26]:
test(800000)

800000
/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate800000_300000_fbuff-1_itr0_dvfs0x0c00_rapl135
0: itr=0 dvfs=0x0c00 num_interrupts=2835166 energy=6986.85 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate800000_300000_fbuff-1_itr300_dvfs0x0c00_rapl135
0: itr=300 dvfs=0x0c00 num_interrupts=905014 energy=7014.22 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate800000_300000_fbuff-1_itr1000_dvfs0x0c00_rapl135
0: itr=1000 dvfs=0x0c00 num_interrupts=551232 energy=7042.1 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate800000_300000_fbuff-1_itr0_dvfs0x1d00_rapl135
0: itr=0 dvfs=0x1d00 num_interrupts=3572209 energy=9927.64 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate800000_300000_fbuff-1_itr300_dvfs0x1d00_rapl135
0: itr=300 dvfs=0x1d00 num_interrupts=934791 energy=9300.85 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate800000_300000_fbuff-1_itr1000_dvfs0x1d00

In [21]:
print(19*60)

1140


In [23]:
print(1140/7)

162.85714285714286


In [24]:
print(162/(3*2))

27.0


In [ ]:
5 itr, 5 dvfs
0 150 300 600 1000
0x0c00 0x0f00 0x1100 0x1900 0x1d00